# Example creation of input river mesh .npz files

*fluvial-particle* uses VTK classes to store the river mesh through which particles move and from which hydrodynamic field data (e.g. depth, velocity) are interpolated. By default, these meshes are expected to be VTK curvilinear mesh files (specifically, the [VTKStructuredGrid() class](https://vtk.org/doc/nightly/html/classvtkStructuredGrid.html#details)) generated by the [iRIC FaSTMECH](https://i-ric.org/en/solvers/fastmech/) river flow solver. To permit the use of curvilinear meshes generated through other methods, *fluvial-particle* will also accept a [NumPy .npz file](https://numpy.org/doc/stable/reference/generated/numpy.savez.html) for the input 2D and 3D river meshes. This notebook is intended to be a guide on the creation of example .npz mesh files for input to a *fluvial-particle* simulations. 

The .npz input file(s) must have arrays for all of the required fields as described below. These fields must be node-centered (as opposed to cell-centered), and the arrays must be ordered in C-order with along-stream mesh nodes stored contiguously, followed by cross-stream nodes, and then by vertical nodes (if 3D). That is, if the number of nodes in the along-stream direction is ns, the number in the cross-stream direction is nn, and the number in the vertical is nz, then each array in the 2D input file must have shape (nn, ns), and each array in the 3D input file must have shape = (nz, nn, ns).

Required vectors for a 2D input mesh must be saved with these exact keys:
* "x": x coordinates of the mesh [m]
* "y": y coordinates of the mesh [m]
* "elev": topographic elevation [m]
* "ibc": indicates whether node is wet (1) or dry (0)
* "shear": shear stress magnitude [N $\cdot$ m<sup>-2</sup>]
* "vx": x-component of fluid velocity [m $\cdot$ s<sup>-1</sup>]
* "vy": y-component of fluid velocity [m $\cdot$ s<sup>-1</sup>]
* "wse": water surface elevation [m]
* "z" (optional): z coordinates of the mesh. Defaults to constant 0. [m]
* "vz" (optional): z-component of fluid velocity. Defaults to constant 0. [m $\cdot$ s<sup>-1</sup>]

Required vectors for a 3D input mesh must be saved with these exact keys:
* "x": x coordinates of the mesh [m]
* "y": y coordinates of the mesh [m]
* "z": z coordinates of the mesh [m]
* "vx": x-component of fluid velocity [m $\cdot$ s<sup>-1</sup>]
* "vy": y-component of fluid velocity [m $\cdot$ s<sup>-1</sup>]
* "vz": z-component of fluid velocity [m $\cdot$ s<sup>-1</sup>]

There are some additional rules that the .npz mesh must be consistent with in order to be compatible with *fluvial-particle*. These are:
1. In the cross-stream direction, there must be at least one dry node at each end of the mesh. That is, ibc=0 must be true for all nodes with cross-stream indices j=0 and j=nn-1. This is required to keep particles from wondering laterally outside the mesh.
2. Particles are expected to exit the modeling domain throught the furthest upstream or downstream boundaries. Particles are only deactivated when they reach cells composed of nodes with along-stream indices i=0 and i=ns-1. Therefore, the river flow must direct particles and allow them to reach one of these indices (generally i=ns-1) in order for them to be correctly deactivated in a simulation.
3. If running a 3D simulation, the x & y coordinates must exactly coincide in both 2D and 3D meshes, with additional vertical nodes in the 3D mesh stacked directly on top of each other. Further, the bottom-most vertical node in the 3D mesh must have z value equal to the *elev* array value at that node, and the upper-most vertical node must coincide with the corresponding *wse* node. This is to prevent particles from entering "gaps" in the water column where data is not defined. See the FM_GridFill.ipynb Jupyter notebook for an example on how to close the gap between topography from a 2D mesh and the bottom-node of a 3D mesh.


In [1]:
import numpy as np
import vtk
from vtk.util import numpy_support

In [2]:
# Load VTK meshes from which to source data
fname_2d = "../tests/data/Result_straight_2d_1.vtk"
fname_3d = "../tests/data/Result_straight_3d_1.vtk"
vgrid_2d = vtk.vtkStructuredGrid()
vgrid_3d = vtk.vtkStructuredGrid()
reader_2d = vtk.vtkStructuredGridReader()
reader_3d = vtk.vtkStructuredGridReader()
reader_2d.SetFileName(fname_2d)
reader_2d.SetOutput(vgrid_2d)
reader_3d.SetFileName(fname_3d)
reader_3d.SetOutput(vgrid_3d)
reader_2d.Update()
reader_3d.Update()

dims_2d = vgrid_2d.GetDimensions()
dims_3d = vgrid_3d.GetDimensions()
print(f"2D mesh dimensions: (ns, nn, nz) = {dims_2d}")  # note that VTK returns mesh dimensions in Fortran-order
print(f"3D mesh dimensions: (ns, nn, nz) = {dims_3d}")
newdims_2d = dims_2d[1::-1]  # returns tuple(nn, ns)
newdims_3d = dims_3d[::-1]  # returns tuple(nz, nn, ns)

2D mesh dimensions: (ns, nn, nz) = (401, 21, 1)
3D mesh dimensions: (ns, nn, nz) = (401, 21, 11)


In [3]:
# Before continuing, it is important to note that the VTK .GetArray() methods return flattened arrays
# The flattened arrays are ordered in the same way as described above (i.e. contiguously along-stream)

# First we'll extract the 2D mesh node coordinates
pts_vtk = vgrid_2d.GetPoints().GetData()
pts_np = numpy_support.vtk_to_numpy(pts_vtk)  # convert the VTK array to a NumPy array, shape (ns*nn, 3)
x = pts_np[:, 0]  # flattened array of x coordinates
y = pts_np[:, 1]  # flattened array of y coordinates
# ignore the z coordinates in this example

# Next, extract the required field data
# These use different keys than what will be used in the .npz file
ptdata = vgrid_2d.GetPointData()
elev = numpy_support.vtk_to_numpy(ptdata.GetArray("Elevation"))
ibc = numpy_support.vtk_to_numpy(ptdata.GetArray("IBC"))
shear = numpy_support.vtk_to_numpy(ptdata.GetArray("ShearStress (magnitude)"))
wse = numpy_support.vtk_to_numpy(ptdata.GetArray("WaterSurfaceElevation"))
vel_vector = numpy_support.vtk_to_numpy(ptdata.GetArray("Velocity"))  # returns (ns*nn, 3) vector
vx = vel_vector[:, 0]
vy = vel_vector[:, 1]
# ignore z components in this example

# Next, reshape the flattened arrays into the correct mesh shape
x = x.reshape(newdims_2d)
y = y.reshape(newdims_2d)
elev = elev.reshape(newdims_2d)
ibc = ibc.reshape(newdims_2d)
shear = shear.reshape(newdims_2d)
wse = wse.reshape(newdims_2d)
vx = vx.reshape(newdims_2d)
vy = vy.reshape(newdims_2d)

# Finally, we're ready to save to a new npz file
np.savez("../tests/data/Result_straight_2d_1.npz", x=x, y=y, elev=elev, ibc=ibc, shear=shear, wse=wse, vx=vx, vy=vy)

In [4]:
# Now we do the same for the 3D mesh

# Extract the 3D mesh node coordinates
pts_vtk = vgrid_3d.GetPoints().GetData()
pts_np = numpy_support.vtk_to_numpy(pts_vtk)  # shape (ns*nn*nz, 3)
x = pts_np[:, 0]
y = pts_np[:, 1]
z = pts_np[:, 2]

# Extract the 3D velocity vector
ptdata = vgrid_3d.GetPointData()
vel_vector = numpy_support.vtk_to_numpy(ptdata.GetArray("Velocity"))
vx = vel_vector[:, 0]
vy = vel_vector[:, 1]
vz = vel_vector[:, 2]

# Reshape into the correct mesh shape
x = x.reshape(newdims_3d)
y = y.reshape(newdims_3d)
z = z.reshape(newdims_3d)
vx = vx.reshape(newdims_3d)
vy = vy.reshape(newdims_3d)
vz = vz.reshape(newdims_3d)

# Save to a new npz file
np.savez("../tests/data/Result_straight_3d_1.npz", x=x, y=y, z=z, vx=vx, vy=vy, vz=vz)

# These meshes can now be used as input to *fluvial-particle* !